In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd
import numpy as np
from IPython.display import display
import time

from keras.layers import Dense, Activation, Dropout, LSTM, Input
from keras.layers import *
from keras.models import Sequential, Model

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

In [ ]:


# Define Date Ranges
start_date = '2000-01-01'
end_date = '2022-12-31'

# Fetch the stock data
ticker_symbol = 'GOOG'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the data
print(stock_data)

# data_stored = train_data

# df = train_data

In [ ]:
df = stock_data.copy();

ema = df['Adj Close'].ewm(com=0.8).mean()
df['EMA'] = ema
df.tail()


change = df["Adj Close"].diff()
change.dropna(inplace=True)
change_up = change.copy()
change_down = change.copy()
change_up[change_up<0] = 0
change_down[change_down>0] = 0
change.equals(change_up+change_down)
avg_up = change_up.rolling(14).mean()
avg_down = change_down.rolling(14).mean().abs()
rsi = 100 * avg_up / (avg_up + avg_down)
df['RSI'] = rsi
df.RSI=df.RSI.fillna(df.RSI.median())
print(df.head(5))


In [ ]:
# train_data = train_data.drop(['High','Low'], axis = 1)
train_data = df.drop(['High','Low','Adj Close'], axis = 1)


In [ ]:
scaler = MinMaxScaler()
# numerical = [['Open', 'Close', 'Adj Close', 'Volume', 'Exponential Moving Average','RSI']]
numerical = ['Close', 'Volume']

train_data[numerical] = scaler.fit_transform(train_data[numerical])

In [ ]:
plt.rcParams['figure.figsize'] = (18, 12)
title = 'Google Trading'
y_label = 'Price USD'
x_label = 'Trading Days'

# Create plot
fig, ax = plt.subplots()
ax.plot(train_data.index, train_data['Close'], color='#0A7388')

# Set title and labels
ax.set_title(title)
ax.set_ylabel(y_label)
ax.set_xlabel(x_label)

# Display the plot
plt.show()


In [ ]:
def unroll(data, sequence_length=24):
    """
    use different windows for testing and training to stop from leak of information in the data
    :param data: data set to be used for unrolling
    :param sequence_length: window length
    :return: data sets with different window.
    """
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)



In [ ]:
prediction_time=5
test_data_size=500 
unroll_length=50

test_data_cut = test_data_size + unroll_length + 1

x_train = train_data[0:-prediction_time - test_data_cut].values
y_train = train_data[prediction_time:-test_data_cut]['Close'].values

# test data
x_test = train_data[0 - test_data_cut:-prediction_time].values
y_test = train_data[prediction_time - test_data_cut:]['Close'].values

x_train.shape, x_test.shape, y_train.shape, y_test.shape




In [ ]:
def unroll(data, sequence_length=24):
    """
    use different windows for testing and training to stop from leak of information in the data
    :param data: data set to be used for unrolling
    :param sequence_length: window length
    :return: data sets with different window.
    """
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)

x_train = unroll(x_train, unroll_length)
x_test = unroll(x_test, unroll_length)


In [ ]:
y_train = y_train[-x_train.shape[0]:]
y_test = y_test[-x_test.shape[0]:]

In [ ]:
x_train, x_test, y_train, y_test

In [ ]:
def build_basic_model(input_dim, output_dim, return_sequences):
    """
    Builds a basic lstm model 
    :param input_dim: input dimension of the model
    :param output_dim: output dimension of the model
    :param return_sequences: return sequence of the model
    :return: a basic lstm model with 3 layers.
    """
    model = Sequential()
    model.add(LSTM(
        input_shape=(None, input_dim),
        units=output_dim,
        return_sequences=return_sequences))

    model.add(LSTM(
        100,
        return_sequences=False))

    model.add(Dense(
        units=1))
    model.add(Activation('linear'))

    return model

In [ ]:


model = build_basic_model(input_dim = x_train.shape[-1],output_dim = unroll_length, return_sequences=True)
start = time.time() 
model.compile(loss='mean_squared_error', optimizer='adam')
print('compilation time : ', time.time() - start)


In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=5,
    validation_split=0.05)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
def plot_lstm_prediction(actual, prediction, title='Google Trading vs Prediction', y_label='Price USD', x_label='Trading Days'):
    """
    Plots train, test and prediction
    :param actual: DataFrame containing actual data
    :param prediction: DataFrame containing predicted values
    :param title:  Title of the plot
    :param y_label: yLabel of the plot
    :param x_label: xLabel of the plot
    :return: prints a Pyplot againts items and their closing value
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # Add labels
    plt.ylabel(y_label)
    plt.xlabel(x_label)

    # Plot actual and predicted close values

    plt.plot(actual, '#00FF00', label='Adjusted Close')
    plt.plot(prediction, '#0000FF', label='Predicted Close')

    # Set title
    ax.set_title(title)
    ax.legend(loc='upper left')


    plt.show()

In [ ]:
plot_lstm_prediction(y_test,predictions)

In [ ]:
trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: %.8f MSE (%.8f RMSE)' % (trainScore, math.sqrt(trainScore)))

testScore = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: %.8f MSE (%.8f RMSE)' % (testScore, math.sqrt(testScore)))

In [ ]:
Train Score: 0.00014250 MSE (0.01193723 RMSE)
Test Score: 0.00212615 MSE (0.04611017 RMSE)
    
//ONLY CLOSE AND VOLUME

Train Score: 0.00013180 MSE (0.01148060 RMSE)
Test Score: 0.00327600 MSE (0.05723637 RMSE)
// Open Close and Volume


Train Score: 0.00012633 MSE (0.01123960 RMSE)
Test Score: 0.00186514 MSE (0.04318730 RMSE)
// Close and Volume only, better model

In [ ]:
def build_improved_model(input_dim, output_dim, return_sequences):
    """
    Builds an improved Long Short term memory model using keras.layers.recurrent.lstm
    :param input_dim: input dimension of model
    :param output_dim: ouput dimension of model
    :param return_sequences: return sequence for the model
    :return: a 3 layered LSTM model
    """
    model = Sequential()
    model.add(LSTM(
        input_shape=(None, input_dim),
        units=output_dim,
        return_sequences=return_sequences))

    model.add(Dropout(0.2))

    model.add(LSTM(
        128,
        return_sequences=False))

    model.add(Dropout(0.2))

    model.add(Dense(
        units=1))
    model.add(Activation('linear'))

    return model

In [ ]:
batch_size = 100
epochs = 8
batch_size = 100
epochs = 5

# build improved lstm model
model = build_improved_model( x_train.shape[-1],output_dim = unroll_length, return_sequences=True)

start = time.time()
#final_model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(loss='mean_squared_error', optimizer='adam')
print('compilation time : ', time.time() - start)

In [ ]:
model.fit(x_train, 
          y_train, 
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_split=0.05
         )

In [ ]:
predictions = model.predict(x_test, batch_size=batch_size)

In [ ]:
plot_lstm_prediction(y_test,predictions)

In [ ]:
trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: %.8f MSE (%.8f RMSE)' % (trainScore, math.sqrt(trainScore)))

testScore = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: %.8f MSE (%.8f RMSE)' % (testScore, math.sqrt(testScore)))

In [ ]:
Train Score: 0.00010789 MSE (0.01038712 RMSE)
Test Score: 0.00953075 MSE (0.09762555 RMSE)

In [ ]:
data_stored

In [ ]:
df = data_stored[['Open','High','Close','Low','Adj Close','Volume']]
df = df.reset_index(drop=True)
length = df.shape[0]
seq_length = 50
step_predict = 5
df['close_log'] = np.log(df['Close'] / df['Close'].shift(1))
df['vol_log'] = np.log(df['Volume'] / df['Volume'].shift(1))
df['target_log'] = np.log(df['Close'].shift(-5) / df['Close'])
df

In [ ]:
x_train

In [ ]:


# train_df = df[1000:3500].copy()
# train_df.reset_index(inplace=True)
# val_df = df[3500:4000].copy()
# val_df.reset_index(inplace=True)
# test_df = df.loc[4000:].copy()
# test_df.reset_index(inplace=True)
# train_df.shape, val_df.shape, test_df.shape


# X_train = []
# y_train = []
# for i in range(seq_length, train_df.shape[0]-step_predict):
#     X_train.append(train_df[i-seq_length+1: i][['close_log', 'vol_log']].values)
#     y_train.append(train_df.loc[i-1, 'target_log'])
# X_train, y_train= np.array(X_train), np.array(y_train)
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
# X_train.shape, y_train.shape


# X_val = []
# y_val = []
# for i in range(seq_length, val_df.shape[0]-step_predict):
#     X_val.append(val_df[i-seq_length+1: i][['close_log', 'vol_log']].values)
#     y_val.append(val_df.loc[i-1, 'target_log'])
# X_val, y_val= np.array(X_val), np.array(y_val)
# X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 2))
# X_val.shape, y_val.shape


# def root_mean_squared_error(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true)))

X_train = x_train
Y_train = y_train
    
inputs = Input(shape=(X_train.shape[1], 2))
lstm1 = LSTM(units=50, return_sequences=True)(inputs)
dropout1 = Dropout(rate=0.5)(lstm1)
lstm2 = LSTM(units=50, return_sequences=True)(dropout1)
dropout2 = Dropout(rate=0.5)(lstm2)
lstm3 = LSTM(units=50)(dropout2)
dropout3 = Dropout(rate=0.5)(lstm3)
fc1 = Dense(units=100)(dropout3)
fc2 = Dense(units=50)(fc1)
fc3 = Dense(units=25)(fc2)
output = Dense(units=1)(fc3)

lstm_model = Model(inputs=inputs, outputs=output)
lstm_model.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = lstm_model.fit(X_train, y_train, epochs = 3, batch_size = 32,validation_split=0.05)

In [ ]:
predictions = lstm_model.predict(x_test)
plot_lstm_prediction(y_test,predictions)

In [ ]:
X_test = []
y_test = []
for i in range(seq_length, test_df.shape[0]-step_predict):
    X_test.append(test_df[i-seq_length+1: i][['close_log', 'vol_log']].values)
    y_test.append(test_df.loc[i-1, 'target_log'])
X_test, y_test= np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))
X_test.shape, y_test.shape


test_price = test_df[seq_length+step_predict:test_df.shape[0]].Close
test_predicted = lstm_model.predict(X_test)
# test_predicted = sc.inverse_transform(test_predicted)
inv = test_df[seq_length:test_df.shape[0]-step_predict].Close.values.reshape(-1, 1)
test_predicted = inv * np.exp(1)**test_predicted
len(test_price), len(test_predicted)

date = test_df[seq_length+step_predict:test_df.shape[0]].index.values
plt.figure(figsize=(20,10))
plt.plot(date, test_price, color = 'red', label = 'Real DJIA')
plt.plot(date, test_predicted, color = 'blue', label = 'Predicted DJIA')
plt.xticks(np.arange(0,len(test_predicted),50))
plt.title('TEST SET: DJIA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('DJIA Stock Price')
plt.legend()
plt.show()
print(f'RMSE: {mean_squared_error(test_price, test_predicted, squared=False)}')